In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir('D:\\ALABS\\ML\\ML ONLINE JAN25 _ Class 7 Machine Learning using Python (Online) JAN 2025')

In [4]:
df = pd.read_csv('test_Loan_Pred - 1739709327694.csv')
df.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


## Null Value Imputation

In [6]:
df.isnull().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [7]:
for col in ['Gender','Dependents','Self_Employed','Loan_Amount_Term','Credit_History']:
    val = df[col].mode().values[0]
    df[col] = df[col].fillna(val)

In [8]:
value = df['LoanAmount'].median()
df['LoanAmount'] = df['LoanAmount'].fillna(value)

In [9]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

## Encoding

In [10]:
mapping_df = pd.read_csv('mapping_dict_train_dataset.csv')
mapping_dict = dict(mapping_df.values)

In [11]:
cat_col = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']
for col in cat_col:
    df[col] = df[col].apply(lambda x: mapping_dict[x])

df.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,0,0,5720,0,110.0,360.0,1.0,2
1,LP001022,1,1,1,0,0,3076,1500,126.0,360.0,1.0,2
2,LP001031,1,1,2,0,0,5000,1800,208.0,360.0,1.0,2


## Standardization

In [12]:
final_df = df['Loan_ID']

In [13]:
df = df.iloc[:,1:]
df.head(3)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,1,0,0,0,5720,0,110.0,360.0,1.0,2
1,1,1,1,0,0,3076,1500,126.0,360.0,1.0,2
2,1,1,2,0,0,5000,1800,208.0,360.0,1.0,2


In [14]:
import pickle

In [15]:
with open('./Standard_Scaler.pkl','rb') as file:
    ss = pickle.load(file)

In [16]:
x_ft = ss.fit_transform(df)
x_ft = pd.DataFrame(x_ft)
x_ft.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.485479,0.758358,-0.758222,-0.544812,-0.334845,0.186461,-0.673335,-0.426780,0.266022,0.437674,1.118764
1,0.485479,0.758358,0.181871,-0.544812,-0.334845,-0.352692,-0.029848,-0.163953,0.266022,0.437674,1.118764
2,0.485479,0.758358,1.121964,-0.544812,-0.334845,0.039641,0.098849,1.183033,0.266022,0.437674,1.118764


## Predicting Final Data

In [18]:
with open('./Best_model.pkl','rb') as file:
    lr = pickle.load(file)

In [19]:
lr

LogisticRegression()

In [21]:
pred = lr.predict(x_ft)

In [22]:
pred_df = pd.DataFrame(pred,columns=['Loan_Status'])
pred_df.head(3)

,Loan_Status
0,1
1,1
2,1


In [24]:
map_dict = {0:'N',1:'Y'}
pred_df['Loan_Status'] = pred_df['Loan_Status'].apply(lambda x: map_dict[x])
pred_df.head(3)

,Loan_Status
0,Y
1,Y
2,Y


In [27]:
submit_df = pd.concat([final_df,pred_df],axis=1)
submit_df.head(3)

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y


In [28]:
submit_df.to_csv('Final_submission.csv',index=False)